# 4.2 MCP 서버의 도구를 활용하는 방법

- 3.8 커스텀 도구(tool)를 최대한 활용하는 방법에서 생성한 에이전트가 활용하는 도구를 [MCP 서버](https://github.com/modelcontextprotocol/servers)로 이전



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

- 도구(tool)을 활용하려면 [`AIMessage`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html)를 활용해야하기 때문에 상태에 `messages`가 필요함
- 직접 선언하지 않고 랭그래프에서 제공하는 `MessagesState`를 사용

In [2]:

from langgraph.graph import StateGraph, MessagesState

class AgentState(MessagesState):
    pass

graph_builder = StateGraph(AgentState)

In [3]:
query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'

In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment='gpt-4o-2024-11-20',
    api_version='2024-08-01-preview',   
    temperature=0
)

- MCP 서버와 연결하기 위한 MCP 클라이언트 생성
    - LangChain에서 제공하는 [langchain-mcp-adapters](https://github.com/langchain-ai/langchain-mcp-adapters)활용
    - `MultiServerMCPClient`를 활용하면 복수의 MCP 서버와 연결할 수 있음

In [5]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:

            key = list(chunk.keys())[0]
            
            if key == 'agent':
                # Agent 메시지의 내용을 가져옴. 메세지가 비어있는 경우 어떤 도구를 어떻게 호출할지 정보를 가져옴
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print(f"'agent': '{content}'")
            
            elif key == 'tools':
                # 도구 메시지의 내용을 가져옴
                for tool_msg in chunk['tools']['messages']:
                    print(f"'tools': '{tool_msg.content}'")
            
            results.append(chunk)
        return results
    except Exception as e:
        print(f"Error processing stream: {e}")
        return results

In [8]:
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.prompts import load_mcp_prompt
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "house_tax": {
            "command": "python",
            "args": ["./mcp_stdio_server.py"],
            "transport": "stdio",
        },
    }
) as client:

    ##### AGENT #####
    tool_list = client.get_tools()
    agent = create_react_agent(llm, tool_list)
    query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'
    print("=====PROMPT=====")
    system_prompt = await client.get_prompt(server_name="house_tax", prompt_name="house_tax_system_prompt", arguments={})
    print("system_prompt : ", system_prompt)
    messages = system_prompt + [HumanMessage(content=query)]
    print("=====RESPONSE=====")
    stream_generator = agent.astream({'messages': [HumanMessage(content=query)]})


    all_chunks = await process_stream(stream_generator)


    if all_chunks:
        final_result = all_chunks[-1]
        print("\nFinal result:", final_result)

=====PROMPT=====
system_prompt :  [HumanMessage(content='당신의 역할은 주택에 대한 종합부동산세를 계산하는 것입니다. \n    사용자의 질문이 들어오면, 사용자의 질문을 바탕으로 종합부동산세를 계산해주세요.\n    종합부동산세를 계산하기 위해서는 과세표준을 어떻게 계산할지 파악해야하고, \n    사용자에 질문에 따른 공제액을 파악해야 하고, \n    사용자에 질문에 따른 공정시장가액비율을 파악해야 합니다.\n    이 세가지를 파악하고 나면, 종합부동산세를 계산해주세요.\n    ', additional_kwargs={}, response_metadata={})]
=====RESPONSE=====


/var/folders/qj/q193qt6n5blc2wppqrq7477r0000gn/T/ipykernel_13516/2116909386.py:23: RuntimeWarning: coroutine 'MultiServerMCPClient.get_prompt' was never awaited
  system_prompt = await client.get_prompt(server_name="house_tax", prompt_name="house_tax_system_prompt", arguments={})


'agent': '{'tool_calls': [{'id': 'call_8yFKmsUpYRvhpl0uDNOXAGb9', 'function': {'arguments': '{}', 'name': 'tax_base_tool'}, 'type': 'function'}, {'id': 'call_m6aNpnBKOGYuwxRVswhCQNyl', 'function': {'arguments': '{"question": "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 공정시장가액비율은 얼마인가요?"}', 'name': 'market_value_rate_tool'}, 'type': 'function'}, {'id': 'call_B7l8y0SYImBroO6VWryDer5F', 'function': {'arguments': '{"question": "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 공제액은 얼마인가요?"}', 'name': 'tax_deductible_tool'}, 'type': 'function'}], 'refusal': None}'
'tools': '과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율'
'tools': '공정시장가액비율은 60%입니다.'
'tools': '9억 원'
'agent': '{'tool_calls': [{'id': 'call_6X3p6y4evfK6ftRNZnEL1PZT', 'function': {'arguments': '{"tax_base_question":"과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율","market_value_rate":"60%","tax_deductible":"9억원","question":"5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?"}', 'name': 'house_tax_tool'}, 'type': 'function'}], 'refusal': None}'
'